## Heart Disease Dataset Variables (with Severity Notes)

* **`ChestPainType`**: Type of chest pain.
    * `TA`: Typical Angina -> 🚨 **High Risk** (Classic heart attack warning)
    * `ATA`: Atypical Angina -> ⚠️ **Moderate Risk**
    * `NAP`: Non-Anginal Pain -> ✅ **Low Risk** (Likely not heart-related)
    * `ASY`: Asymptomatic -> ❓ **Silent/Sneaky** (Often dangerous in ML models)
* **`RestingBP`**: Resting blood pressure [mm Hg].
    * *Note:* Higher values -> ⚠️ **Worse** (Hypertension strain)
* **`Cholesterol`**: Serum cholesterol [mm/dl].
    * *Note:* Higher values -> ⚠️ **Worse** (Risk of blockage)
* **`FastingBS`**: Fasting blood sugar.
    * `0`: Normal (< 120 mg/dl) -> ✅ **Good**
    * `1`: High (> 120 mg/dl) -> ⚠️ **Diabetes Risk** (Damages vessels)
* **`RestingECG`**: Resting electrocardiogram results.
    * `Normal`: Normal -> ✅ **Good**
    * `ST`: ST-T abnormality -> ⚠️ **Bad** (Ischemia/Oxygen lack)
    * `LVH`: Left Ventricular Hypertrophy -> 🚨 **Severe** (Thickened heart muscle)
* **`MaxHR`**: Maximum heart rate achieved.
    * *Note:* Lower max achievable HR often indicates -> ⚠️ **Weaker Heart**
* **`ExerciseAngina`**: Exercise-induced angina.
    * `N`: No -> ✅ **Good**
    * `Y`: Yes -> 🚨 **Critical Flag** (Heart fails under stress)
* **`Oldpeak`**: ST depression induced by exercise vs rest.
    * *Note:* Higher value (>1.0) -> 🚨 **Tragic/Severe Ischemia**
* **`ST_Slope`**: The slope of the peak exercise ST segment.
    * `Up`: Upsloping -> ✅ **Healthy Response**
    * `Flat`: Flat -> ⚠️ **Warning**
    * `Down`: Downsloping -> ☠️ **Worst/Tragic** (Strongest sign of disease)

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from heart_failure_prediction.config import RAW_DATA_DIR

In [ ]:
path = os.path.join(RAW_DATA_DIR, 'heart.csv')
heart = pd.read_csv(path)

In [ ]:
heart.head()

### Encoding binary values as 0/1

In [ ]:
heart['ExerciseAngina'] = (
    heart['ExerciseAngina'].replace({'N': '0', 'Y': '1'}).astype(int)
)
heart['Sex'] = heart['Sex'].replace({'M': '0', 'F': '1'}).astype(int)

### Data types

In [ ]:
heart.dtypes

### Missing and duplicated values

In [ ]:
print(f'Null value count: {heart.isna().sum().sum()}')

In [ ]:
print(f'Duplicated values count: {heart.duplicated().sum()}')

### Data distribution

In [ ]:
cols = heart.columns

n_cols = 4
n_rows = int(np.ceil(len(cols) / n_cols))

plt.figure(figsize=(3 * n_cols, 2 * n_rows))

for i, col in enumerate(cols, start=1):
    plt.subplot(n_rows, n_cols, i)
    heart[col].hist(bins=50)
    plt.title(col)
    plt.tight_layout()

In [ ]:
heart['HeartDisease'].value_counts(normalize=True) * 100

Cholesterol can't be 0! Suggests missing values.

In [ ]:
heart['Cholesterol'] = heart['Cholesterol'].replace({0: np.nan})

In [ ]:
print(f'Null value count: {heart.isna().sum().sum()}')

In [ ]:
heart['col_missing'] = heart['Cholesterol'].isna().astype(int)
print(heart.groupby('col_missing')['HeartDisease'].mean())
heart.drop('col_missing', axis=1, inplace=True)

Missing cholesterol suggests critical status of patient! Should add indicator while imputing later on.

### Correlation

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    heart.select_dtypes(np.number).corr(), annot=True, fmt='.2f', cmap='coolwarm'
)
plt.show()

In [ ]:
sns.countplot(x='ChestPainType', hue='HeartDisease', data=heart)

In [ ]:
sns.countplot(x='RestingECG', hue='HeartDisease', data=heart)

In [ ]:
sns.countplot(x='ST_Slope', hue='HeartDisease', data=heart)